In [1]:
using Circulation
using GPFields
using TimerOutputs

In [2]:
const DATA_DIR_BASE = let dir = expanduser("~/Dropbox/circulation/data")
    @show readdir(dir)
    # joinpath(dir, "4vortices")
    # joinpath(dir, "TG")
    joinpath(dir, "tangle")
end

const DATA_IDX = 1
const RESOLUTION = 256

const GP_PARAMS = ParamsGP(
    (RESOLUTION, RESOLUTION, RESOLUTION),
    c = 1.0,
    nxi = 1.5,
)

let dir = joinpath(DATA_DIR_BASE, string(RESOLUTION))
    println("Directories in $DATA_DIR_BASE: ", readdir(DATA_DIR_BASE))
    println("Files in $dir: ", readdir(dir))
end

readdir(dir) = ["4vortices", "KW", "TG", "TG2D", "rotation", "tangle"]
Directories in /home/jipolanco/Dropbox/circulation/data/tangle: ["128", "256"]
Files in /home/jipolanco/Dropbox/circulation/data/tangle/256: ["ImaPsi.001.dat", "ReaPsi.001.dat", "tt.001.dat"]


In [6]:
N = GP_PARAMS.dims[1]
# loop_sizes = (1, 2, 4, 8, 16)
power_max = Int(log2(N)) - 1
loop_sizes = [1 << n for n = 0:power_max]  # powers of two
@show loop_sizes
to = TimerOutput()

κ = GP_PARAMS.κ  # not sure if this is the good κ...

stats = Circulation.init_statistics(
    loop_sizes,
    hist_edges=LinRange(-10κ, 10κ, 500),
    num_moments=20,
)

@time analyse!(stats, GP_PARAMS, DATA_DIR_BASE,
    data_idx=DATA_IDX, eps_vel=0, to=to,
    max_slices=2,  # to make things faster! (testing only)
);

loop_sizes = [1, 2, 4, 8, 16, 32, 64, 128]


Slice(x): 100%|█████████████████████████████████████████| Time: 0:00:00
Slice(y): 100%|█████████████████████████████████████████| Time: 0:00:00
Slice(z): 100%|█████████████████████████████████████████| Time: 0:00:00


  1.353199 seconds (4.59 k allocations: 24.591 MiB)


In [7]:
save_statistics("stats.h5", stats)

HFD5 data file (closed): stats.h5

In [8]:
println(to)

 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            3.30s / 41.0%           25.3MiB / 97.3%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 analyse!                   3    1.35s   100%   451ms   24.6MiB  100%   8.20MiB
   circulation!           144    678ms  50.1%  4.71ms     0.00B  0.00%    0.00B
   statistics             144    525ms  38.8%  3.65ms   6.16KiB  0.02%    43.8B
     histogram            144    314ms  23.2%  2.18ms     0.00B  0.00%    0.00B
     moments              144    210ms  15.5%  1.46ms     0.00B  0.00%    0.00B
   prepare!                18   65.5ms  4.84%  3.64ms     0.00B  0.00%    0.00B
   load_psi!                6   45.8ms 